In [ ]:
#| default_exp interactive

# Interactive

> Interactive components that require more elaborate implementations

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.utils import *
from fhbasecoat.common import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from itertools import product
from enum import Enum, auto


In [ ]:
app = FastHTML(session_cookie="mysession")
rt = app.route

In [ ]:
# srv.stop()

In [ ]:
srv = JupyUvi(app)

In [ ]:
from pathlib import Path
common_components = Path("../fhbasecoat/common.py").read_text()

When answering questions refer to already made common components with $`common_components`

### Dialog / Alert Dialog

In [ ]:
#| export
def DialogOpenButton(*args, did, cls=ButtonT.outline, **kwargs):
    return Button(*args, onclick=f"document.getElementById('{did}').showModal()", cls=cls, **kwargs)

In [ ]:
p(DialogOpenButton("Open", did="d1"))

In [ ]:
#| export
def DialogCloseButton(content=Icon("x"), cls="", **kwargs):
    return Button(content, onclick="this.closest('dialog').close()", cls=cls, aria_label="Close dialog", **kwargs)

Note: we're defaulting this to no button styling because it recieves some base style when inside the Dialog element.

In [ ]:
p(DialogCloseButton())

In [ ]:
#| export
def Dialog(*contents, id, title=None, desc=None, footer=None, close_btn=DialogCloseButton(), cls="w-full sm:max-w-[425px] max-h-[612px]", onclick="if (event.target === this) this.close()", **kwargs):
    parts = []
    if title or desc: parts.append(Header(H2(title), P(desc)))
    parts.append(Section(*contents))
    if footer: parts.append(Footer(footer))
    return fh.Dialog(Div(*parts, close_btn), id=id, cls=f"dialog {cls}", onclick=onclick, **kwargs)

In [ ]:
pw(
    DialogOpenButton("Edit Profile", did="dia1"),
    Dialog(
        Form(
            Div(
                Label("Name", fr="name"),
                Input(id="name", value="Pedro Duarte", autofocus=True, type="text"),
                cls="grid gap-3"
            ),
            Div(
                Label("Username", fr="username"),
                Input(id="username", value="@peduarte", type="text"),
                cls="grid gap-3"
            ),
            cls="grid gap-4"
        ),
        id="dia1",
        title="Edit profile",
        desc="Make changes to your profile here. Click save when you're done.",
        footer=Div(
            Button("Cancel", onclick="this.closest('dialog').close()", cls=ButtonT.outline),
            Button("Save changes", onclick="this.closest('dialog').close()"),
            cls="flex gap-2"
        )
    )
)

In [ ]:
#| export
@delegates(Dialog, keep=True)
def AlertDialog(*args, close_btn=None, onclick=None, **kwargs):
    return Dialog(*args, close_btn=close_btn, onclick=onclick, **kwargs)

Alert dialog is simply a dialog that requires you to click an action button to close (no 'x' button, and no background click close).

In [ ]:
pw(
    DialogOpenButton("Delete account", did="dia2"),
    AlertDialog(
        id="dia2",
        title="Are you absolutely sure?",
        desc="This action cannot be undone. This will permanently delete your account and remove your data from our servers.",
        footer=Div(
            Button("Cancel", onclick="this.closest('dialog').close()", cls=ButtonT.outline),
            Button("Save changes", onclick="this.closest('dialog').close()"),
            cls="flex gap-2"
        )
    )
)

### Dropdown menu

In [ ]:
#| export
def DropdownTriggerButton(*contents, did, cls=ButtonT.outline, **kwargs):
    return Button(*contents, id=f"{did}-trigger", aria_haspopup="menu", aria_controls=f"{did}-menu", aria_expanded="false", cls=cls, **kwargs)

In [ ]:
#| export
def Separator(role="separator", **kwargs):
    return Hr(role=role, **kwargs)

In [ ]:
#| export
def DropdownHeader(*contents, **kwargs):
    return Div(*contents, role="heading", **kwargs)

In [ ]:
#| export
def DropdownItem(*contents, icon=None, shortcut=None, disabled=False, **kwargs):
    parts = []
    if icon: parts.append(icon)
    parts.extend(contents)
    if shortcut: parts.append(Span(shortcut, cls="text-muted-foreground ml-auto text-xs tracking-widest"))
    return Div(*parts, role="menuitem", aria_disabled="true" if disabled else None, **kwargs)

In [ ]:
#| export
def Dropdown(*contents, id, btn_content="Open", trigger_btn=None, cls="min-w-65", side="bottom", align="left"):
    """The dropdown menu requires a DropdownTriggerButton to activate. A default version is provided used btn_content value
    but can be overriden by passing a button to the trigger_btn param.
    """ 
    return Div(
        trigger_btn or DropdownTriggerButton(btn_content, did=id),
        Div(
            Div(
                *contents,
                role="menu", id=f"{id}-menu", aria_labelledby=f"{id}-trigger",
            ),
            id=f"{id}-popover", data_popover=True, aria_hidden="true", cls=cls,
            data_side=side, data_align=align,
        ),
        id=id, cls="dropdown-menu"
    )


In [ ]:
pw(
    Dropdown(
        DropdownHeader("My Account", id="account-options"),
        DropdownItem("Profile", shortcut="⇧⌘P", icon=Icon("user")),
        DropdownItem("Billing", shortcut="⌘B", icon=Icon("credit-card")),
        Separator(),
        DropdownItem("Github"),
        DropdownItem("Support"),
        DropdownItem("API", disabled=True),
        Separator(),
        DropdownItem("Logout", shortcut="⇧⌘P"),
        id="testing", side="right", align="center",
    )
)

<!-- Sidenote: how do make a bash alias in .bashrc of two different commands in seuqnece -->